In [1]:
import pandas as pd
import time 
import pickle
import os
#nltk.download('punkt')


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import warnings 
warnings.filterwarnings("ignore")

In [2]:
year = "2010"
file_path = "/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/"
file_path_2 = "_final_rnn.pickle"

df_10= pd.read_pickle(file_path + year + file_path_2)

In [3]:
def absolute_count(male_col, female_col):
    if female_col > male_col and male_col == 0:
        return 1
    elif male_col> female_col and female_col ==0: 
        return 0
    else: 
        return None

In [4]:
#apply function to only get rows with an absolute count 
df_10['col_type'] = df_10.apply(lambda row: absolute_count(row['male_count'], row['female_count']),axis=1)

#remove nulls 
df_10 = df_10[df_10["col_type"].notnull()]

#DOC: number of male and female columns
df_10["col_type"].value_counts()  

0.0    378476
1.0    113870
Name: col_type, dtype: int64

**Define TFIDF Vectorizer**

In [5]:
#tfidf vectorizer
def fake(token):
    return token

tfidf = TfidfVectorizer(
    analyzer='word',
    tokenizer=fake,
    preprocessor=fake,
    token_pattern=None)  

**Logistic Regression Classifier**

*What the LR model does-* LR estimates the probability of an instance belonging to the positive class. 

In [6]:
def logistic_regression_year(df, target_col, text_col):
    #start timer 
    start_time = time.time()
    
    #split data 
    X = df[text_col].apply(lambda x: str(x))
    y = df[target_col]

    #train test split
    tfidf = TfidfVectorizer()
    X_transformed = tfidf.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

    # define the hyperparameters to search over
    param_grid = {
        'penalty': ['l1', 'l2'],
        'C': [0.1, 1, 10, 100],
        'solver': ['lbfgs', 'newton-cg', 'sag' 'saga'], #removed liblinear as it is for small + medium datasets & NOT for sparse data
        'class_weight': ['balanced', {0: 0.3, 1: 0.7}],
        'random_state': [42]
    }

    #the classifier 
    clf = LogisticRegression()

    #create a GridsearchCV object 
    grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    best_clf = LogisticRegression(**grid_search.best_params_)

    #run the classifier 
    best_clf.fit(X_train, y_train)
    y_pred = best_clf.predict(X_test)

    #performance 
    accuracy = best_clf.score(X_test, y_test) #evaluate on test set
    class_report = classification_report(y_test, y_pred, zero_division = 0)
    #results = {'accuracy': accuracy, 'classification_report': class_report}
    #print(f"Accuracy: {accuracy:.2f}")
    #print(f"Classification Report:\n{class_report}")

    #coefficients
    coefs = best_clf.coef_[0]
    sorted_coef = sorted((zip(tfidf.get_feature_names_out(), coefs)), key = lambda x: x[1], reverse=True)
    high_coef = sorted_coef[:1000]
    low_coef = sorted_coef[-1000:]
    
    df_high_coef = pd.DataFrame(high_coef, columns=['feature', 'coef'])
    df_low_coef = pd.DataFrame(low_coef, columns=['feature', 'coef'])

    #save model 
    with open('results.pkl', 'wb') as f:
        pickle.dump({'model': best_clf, 'tfidf': tfidf, 'accuracy': accuracy, 'report': class_report}, f)

    #end timer 
    end_time = time.time()
    print(f"\nExecution time: {end_time - start_time:.2f} seconds")

    return df_high_coef, df_low_coef, best_clf, class_report #df_probs_top

**Coefficient Analysis**

In [7]:
df_probs_10 = logistic_regression_year(df_10, 'col_type', 'pre_processed_sent')
df_probs_10


Execution time: 525.98 seconds


(       feature      coef
 0     pregnant  3.186218
 1         baby  3.085429
 2        child  2.502925
 3    boyfriend  2.471323
 4          rap  2.215964
 ..         ...       ...
 995     openly  0.201524
 996     duress  0.201428
 997   campaign  0.201339
 998      nanny  0.200883
 999      susan  0.200859
 
 [1000 rows x 2 columns],
         feature      coef
 0     reinstate -0.213403
 1          bird -0.214428
 2        dinghy -0.214939
 3         evade -0.215387
 4       embassy -0.215427
 ..          ...       ...
 995    football -1.268172
 996  girlfriend -1.327853
 997       shoot -1.620356
 998     soldier -1.622398
 999      arrest -1.826027
 
 [1000 rows x 2 columns],
 LogisticRegression(C=0.1, class_weight={0: 0.3, 1: 0.7}, random_state=42,
                    solver='newton-cg'),
 '              precision    recall  f1-score   support\n\n         0.0       0.81      0.90      0.85     75430\n         1.0       0.48      0.30      0.37     23040\n\n    accuracy         

*Interpreting Performance*

The LR model is WAY better in terms of precision, recall, and f1-score at predicting the negative class - i.e. male. 

In [8]:
#open model performance metrics 
with open('results_10.pkl', 'rb') as f:
    results = pickle.load(f)

accuracy = results['accuracy']
report = results['report']

print(f"Accuracy: {accuracy:.2f}")
print(f"Classification report:\n{report}")

Accuracy: 0.76
Classification report:
              precision    recall  f1-score   support

         0.0       0.81      0.90      0.85     75430
         1.0       0.48      0.30      0.37     23040

    accuracy                           0.76     98470
   macro avg       0.64      0.60      0.61     98470
weighted avg       0.73      0.76      0.74     98470



In [ ]:
#create DF of highest coef
highest_coef_22 = pd.DataFrame(df_probs_22[0])
highest_coef_22["coef_type"] = "highest"
highest_coef_22["year"] = year

#create DF of lowest lowest coef manipulation 
lowest_coef_22 = pd.DataFrame(df_probs_22[1]) 
lowest_coef_22 = lowest_coef_22.sort_values(by = ["coef"], ascending = True).reset_index(drop = True) #absolute lowest value 
lowest_coef_22["coef_type"] = "lowest" #coef type
lowest_coef_22["year"] = year #year 
highest_coef_22

In [ ]:
#save DF as pickle file per year 
lowest_coef_22.to_pickle('RESULTS22_coef_low.pickle')
highest_coef_22.to_pickle('RESULTS22_coef_high.pickle')

**Interpretation of coefficient results**
"winner" is one of the independent variables in the model and its coefficient value is 1.2865737872946597. This means that a one unit increase in the value of the "winner" variable will increase the log-odds of the positive class (e.g. "female" if the logistic regression model is binary and predicting gender) by the corresponding coefficient value, while holding all other variables constant.

**Interpretation of the predicted probabilities in LR:** 
The predicted probabilities of the logistic regression model tell us the probability that the input data belongs to the positive class - in this case the female class as we attributed it a value = 1 in binary log reg. Hence, for each word, we get a list of a word/feature and the probability that it is female. 

These predicted probabilities can be interpreted as the confidence level of the model in its prediction. For example, a predicted probability of 0.8 for a positive class means that the model is 80% confident that the sample belongs to the positive class. 

In [ ]:
def load_pickle_files_low(directory):
    objects = []
    for filename in os.listdir(directory):
        if filename.endswith("low.pickle"):
            with open(os.path.join(directory, filename), 'rb') as file:
                obj = pickle.load(file)
                objects.append(obj)
    return objects

def load_pickle_files_high(directory):
    objects = []
    for filename in os.listdir(directory):
        if filename.endswith("high.pickle"):
            with open(os.path.join(directory, filename), 'rb') as file:
                obj = pickle.load(file)
                objects.append(obj)
    return objects

In [ ]:
#stack dfs together for low and high coefs 
df_low = load_pickle_files_low(r"/Users/yolandaferreirofranchi/Documents/GitHub/Masters-Thesis")
df_high = load_pickle_files_high(r"/Users/yolandaferreirofranchi/Documents/GitHub/Masters-Thesis")
df_high